# Aprendizagem Automática Avançada
## Assignment 5
### Luís Ferreirinha Nº51127
### Christopher Anaya Nª60566

# Problem 1

In [3]:
from pomegranate import *

In [8]:
node_a = DiscreteDistribution({'T': 0.3, 'F':0.7})
node_b = ConditionalProbabilityTable(
          [['F','T',0.4],
           ['F','F',0.6],
           ['T','T',0.8],
           ['T','F',0.2]], [node_a])
node_c = ConditionalProbabilityTable(
      [['F','F','T', 0.1],
       ['F','F','F', 0.9],
       ['F','T','T', 0.7],
       ['F','T','F', 0.3],
       ['T','F','T', 0.5],
       ['T','F','F', 0.5],
       ['T','T','T', 0.99],
       ['T','T','F', 0.01]], [node_a, node_b])
node_d = ConditionalProbabilityTable(
    [['F','T', 0.55],
     ['F','F', 0.45],
     ['T','T', 0.2],
     ['T','F', 0.8]],[node_b])

s_a = Node(node_a, name="a")
s_b = Node(node_b, name="b")
s_c = Node(node_c, name="c")
s_d = Node(node_d, name="d")

model = BayesianNetwork("Problem 1")
model.add_states(s_a, s_b, s_c, s_d)
model.add_edge(s_a, s_b)
model.add_edge(s_a, s_c)
model.add_edge(s_b, s_c)
model.add_edge(s_b, s_d)

model.bake()

a) $P(A = T | C = T, D = T)$

In [13]:
#print(model.predict_proba({'C':'T', 'D':'T'}))
print(model.predict_proba([[None, None, 'T', 'T']]))

[array([{
           "class" : "Distribution",
           "dtype" : "str",
           "name" : "DiscreteDistribution",
           "parameters" : [
               {
                   "T" : 0.5054138717420109,
                   "F" : 0.49458612825798914
               }
           ],
           "frozen" : false
       }                                    ,
       {
           "class" : "Distribution",
           "dtype" : "str",
           "name" : "DiscreteDistribution",
           "parameters" : [
               {
                   "T" : 0.6483149049313832,
                   "F" : 0.3516850950686168
               }
           ],
           "frozen" : false
       }                                    , 'T', 'T'], dtype=object)]


b) $P(A = T | D = F)$

In [14]:
print(model.predict_proba([['T', None, None, 'F']]))

[array(['T', {
                "class" : "Distribution",
                "dtype" : "str",
                "name" : "DiscreteDistribution",
                "parameters" : [
                    {
                        "T" : 0.8767123287671229,
                        "F" : 0.123287671232877
                    }
                ],
                "frozen" : false
            }                                    ,
       {
           "class" : "Distribution",
           "dtype" : "str",
           "name" : "DiscreteDistribution",
           "parameters" : [
               {
                   "T" : 0.9295890410958901,
                   "F" : 0.0704109589041099
               }
           ],
           "frozen" : false
       }                                    , 'F'], dtype=object)]


c) $P(B=T | C = T)$

In [16]:
print(model.predict_proba([[None, None, 'T', None]]))

[array([{
           "class" : "Distribution",
           "dtype" : "str",
           "name" : "DiscreteDistribution",
           "parameters" : [
               {
                   "T" : 0.5253830090147861,
                   "F" : 0.4746169909852139
               }
           ],
           "frozen" : false
       }                                    ,
       {
           "class" : "Distribution",
           "dtype" : "str",
           "name" : "DiscreteDistribution",
           "parameters" : [
               {
                   "T" : 0.8100843263425553,
                   "F" : 0.1899156736574448
               }
           ],
           "frozen" : false
       }                                    , 'T',
       {
           "class" : "Distribution",
           "dtype" : "str",
           "name" : "DiscreteDistribution",
           "parameters" : [
               {
                   "T" : 0.2664704857800823,
                   "F" : 0.7335295142199177
               }
           

d) $P(B=T | A=T, C=T)$

In [17]:
print(model.predict_proba([['T', None, 'T', None]]))

[array(['T', {
                "class" : "Distribution",
                "dtype" : "str",
                "name" : "DiscreteDistribution",
                "parameters" : [
                    {
                        "T" : 0.8878923766816139,
                        "F" : 0.11210762331838604
                    }
                ],
                "frozen" : false
            }                                    , 'T',
       {
           "class" : "Distribution",
           "dtype" : "str",
           "name" : "DiscreteDistribution",
           "parameters" : [
               {
                   "T" : 0.23923766816143516,
                   "F" : 0.7607623318385648
               }
           ],
           "frozen" : false
       }                                     ], dtype=object)]


e) $P(C=T | A=F, B=F, D=F)$

In [18]:
print(model.predict_proba([['F', 'F', None, 'F']]))

[array(['F', 'F', {
                     "class" : "Distribution",
                     "dtype" : "str",
                     "name" : "DiscreteDistribution",
                     "parameters" : [
                         {
                             "T" : 0.10000000000000016,
                             "F" : 0.8999999999999999
                         }
                     ],
                     "frozen" : false
                 }                                     , 'F'],
      dtype=object)]


# Problem 2

In [47]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import cross_val_score
iris = datasets.load_iris()

X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

a) 

In [44]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
predictions = gnb.predict(X_test)
print("GaussianNB:")
print(classification_report(y_test, predictions))

GaussianNB:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       0.93      0.93      0.93        15
           2       0.94      0.94      0.94        16

    accuracy                           0.96        50
   macro avg       0.96      0.96      0.96        50
weighted avg       0.96      0.96      0.96        50



b)

In [49]:
grid_param = [
    {"n_estimators" : [10,50,100], "criterion" : ["gini", "entropy"], "max_depth" : [5,10,15,20],
     "min_samples_split" : [5,10,15,20], "min_samples_leaf" : [5,10,15,20]}
]

grid_search = GridSearchCV(RandomForestClassifier(), grid_param, scoring="", cv=5, n_jobs=-1)

grid_search.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan n

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid=[{'criterion': ['gini', 'entropy'],
                          'max_depth': [5, 10, 15, 20],
                          'min_samples_leaf': [5, 10, 15, 20],
                          'min_samples_split': [5, 10, 15, 20],
                          'n_estimators': [10, 50, 100]}],
             scoring='f1')

In [50]:
print("Best Parameters found:")
print(grid_search.best_params_)

Best Parameters found:
{'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 10}


In [51]:
predictions_b = grid_search.predict(X_test)
print(classification_report(y_test, predictions_b))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       0.94      1.00      0.97        15
           2       1.00      0.94      0.97        16

    accuracy                           0.98        50
   macro avg       0.98      0.98      0.98        50
weighted avg       0.98      0.98      0.98        50

